Purpose: Create our train/test dataset.
- create "False" examples
- match date/location data to weather and fire history data

Take 4 datasets (LDC, GHCN, Kaggle, CalFire); merge into 1

In [ ]:
!pip install shapely
!pip install python-geohash

In [ ]:
import csv
# To validate lat/lon in CA
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.point import Point
import json

import random
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
import pandas as pd
import math
import numpy as np
import geohash


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## MAIN and Globals

In [ ]:
# globals
CAL_FIRE_FILENAME = "/content/drive/My Drive/ML6140  - Project/Raw Data/Cal Fire/CalFire_IncidentData_Reformatted.csv"
LCD_STATIONS_FILE = '/content/drive/My Drive/ML6140  - Project/Raw Data/LCD_STATION_DATA/LCD_MY_STATIONS_ALL_INFO.csv'
GHCN_STATIONS_FILE = '/content/drive/My Drive/ML6140  - Project/Raw Data/NOAA_GHCN-Daily/melio-ca-stations-2013-2023.csv'
LCD_STATIONS_FOLDER = '/content/drive/My Drive/ML6140  - Project/Raw Data/LCD_STATION_DATA/Raw_data_download_mod_preprocessed_converted'
GHCN_STATIONS_FOLDER = '/content/drive/My Drive/ML6140  - Project/Raw Data/NOAA_GHCN-Daily/ca-2013-2023-station-data-reformatted-filtered-units-converted'

EXAMPLE_DATA_START_YEAR = 2015  # Don't allow examples before this year

# dataset we're creating (reading in positive examples, creating negative ones)
# Basic = [year, month, day, hour, minute, latitude, longitude, fire/no fire]
OUR_DATASET_BASIC_FILENAME = "/content/drive/My Drive/ML6140  - Project/Data/basic_data.csv"
# Enhanced = all info from basic, plus weather measurements and other data
OUR_DATASET_ENHANCED_FILENAME = "/content/drive/My Drive/ML6140  - Project/Data/enhanced_data.csv"

BASIC_DATA_FEATURES = [
    'incident_name',
    'incident_created_year',
    'incident_created_month',
    'incident_created_day',
    'incident_created_hour',
    'incident_created_minute',
    'incident_latitude',
    'incident_longitude',
    'class_label']

FEATURE_INCLUSIONS = {
    "incident_geohash": False,
    "station_details": False,
    "daily_weather": False,
    "agg_10_weather": False,
    "agg_30_weather": False,
    "agg_60_weather": False,
    "fire_history": False
}

FEATURE_HEADERS = {
    "incident_geohash": ['incident_geohash'],
    "station_details": ['LCD_station_id',
                        'LCD_station_elevation',
                        'LCD_station_distance',
                        'GHCN_station_id',
                        'GHCN_station_elevation',
                        'GHCN_station_distance'],
    "daily_weather": ['sumPrecipitation', 'dist_sumPrecipitation',
                      'maxDryBulbTemperature', 'dist_maxDryBulbTemperature',
                      'minDryBulbTemperature', 'dist_minDryBulbTemperature',
                      'meanDryBulbTemperature', 'dist_meanDryBulbTemperature',
                      'meanDewPointTemperature', 'dist_meanDewPointTemperature',
                      'meanWetBulbTemperature', 'dist_meanWetBulbTemperature',
                      'meanWindSpeed', 'dist_meanWindSpeed',
                      'meanRelativeHumidity', 'dist_meanRelativeHumidity',
                      'minRelativeHumidity', 'dist_minRelativeHumidity',
                      'maxRelativeHumidity', 'dist_maxRelativeHumidity',
                      'SNOW', 'dist_SNOW',
                      'SNWD', 'dist_SNWD',
                      'EVAP', 'dist_EVAP',
                      'FMTM', 'dist_FMTM',
                      'FRGB', 'dist_FRGB',
                      'FRGT', 'dist_FRGT',
                      'FRTH', 'dist_FRTH',
                      'maxWindSpeed', 'dist_maxWindSpeed',
                      'calculate_circular_meanWindDirection', 'dist_calculate_circular_meanWindDirection',
                      'mode_functionWindDirection', 'dist_mode_functionWindDirection',
                      'maxWindGustSpeed', 'dist_maxWindGustSpeed',
                      'mode_functionPresentWeatherType', 'dist_mode_functionPresentWeatherType',
                      'mode_functionSkyConditions', 'dist_mode_functionSkyConditions'],
    "agg_10_weather": ['10_sumPrecipitation',
                      '10_maxDryBulbTemperature',
                      '10_minDryBulbTemperature',
                      '10_meanDryBulbTemperature',
                      '10_meanDewPointTemperature',
                      '10_meanWetBulbTemperature',
                      '10_meanWindSpeed',
                      '10_meanRelativeHumidity',
                      '10_minRelativeHumidity',
                      '10_maxRelativeHumidity',
                      '10_SNOW',
                      '10_SNWD',
                      '10_EVAP',
                      '10_FMTM',
                      '10_FRGB',
                      '10_FRGT',
                      '10_FRTH',
                      '10_maxWindSpeed',
                      '10_calculate_circular_meanWindDirection',
                      '10_mode_functionWindDirection',
                      '10_maxWindGustSpeed',
                      '10_mode_functionPresentWeatherType',
                      '10_mode_functionSkyConditions'],
    "agg_30_weather": ['30_sumPrecipitation',
                      '30_maxDryBulbTemperature',
                      '30_minDryBulbTemperature',
                      '30_meanDryBulbTemperature',
                      '30_meanDewPointTemperature',
                      '30_meanWetBulbTemperature',
                      '30_meanWindSpeed',
                      '30_meanRelativeHumidity',
                      '30_minRelativeHumidity',
                      '30_maxRelativeHumidity',
                      '30_SNOW',
                      '30_SNWD',
                      '30_EVAP',
                      '30_FMTM',
                      '30_FRGB',
                      '30_FRGT',
                      '30_FRTH',
                      '30_maxWindSpeed',
                      '30_calculate_circular_meanWindDirection',
                      '30_mode_functionWindDirection',
                      '30_maxWindGustSpeed',
                      '30_mode_functionPresentWeatherType',
                      '30_mode_functionSkyConditions'],
    "agg_60_weather": ['60_sumPrecipitation',
                      '60_maxDryBulbTemperature',
                      '60_minDryBulbTemperature',
                      '60_meanDryBulbTemperature',
                      '60_meanDewPointTemperature',
                      '60_meanWetBulbTemperature',
                      '60_meanWindSpeed',
                      '60_meanRelativeHumidity',
                      '60_minRelativeHumidity',
                      '60_maxRelativeHumidity',
                      '60_SNOW',
                      '60_SNWD',
                      '60_EVAP',
                      '60_FMTM',
                      '60_FRGB',
                      '60_FRGT',
                      '60_FRTH',
                      '60_maxWindSpeed',
                      '60_calculate_circular_meanWindDirection',
                      '60_mode_functionWindDirection',
                      '60_maxWindGustSpeed',
                      '60_mode_functionPresentWeatherType',
                      '60_mode_functionSkyConditions'],
    "fire_history": ["far_hist_avg_acres_burned", "near_hist_avg_acres_burned"]
}

BASIC_DATA_HEADER_TO_INDEX = {
    'incident_name': 0,
    'incident_created_year': 1,
    'incident_created_month': 2,
    'incident_created_day': 3,
    'incident_created_hour': 4,
    'incident_created_minute': 5,
    'incident_latitude': 6,
    'incident_longitude': 7,
    'class_label': 8
}

CAL_FIRE_HEADER_TO_INDEX = {
    'incident_name': 0,
    'incident_created_year': 1,
    'incident_created_month': 2,
    'incident_created_day': 3,
    'incident_created_hour': 4,
    'incident_created_minute': 5,
    'incident_acres_burned': 6,
    'incident_latitude': 7,
    'incident_longitude': 8,
    'incident_extinguished_year': 9,
    'incident_extinguished_month': 10,
    'incident_extinguished_day': 11,
    'incident_extinguished_hour': 12,
    'incident_extinguished_minute': 13
}

# From proposal.
# How many negative examples there should be to postive ones.
RATIO_OF_NEGATIVE_EXAMPLES_TO_POSITIVE = 2
# Of the negative examples, this many should have their locations chosen to
# match fire locations.
# This times RATIO_OF_NEGATIVE_EXAMPLES_TO_POSITIVE should be less than 1.
PORTION_OF_NEGATIVE_EXAMPLES_TO_MATCH_FIRE_LOCATIONS = .25

# How close do two points have to be when determining if a negative example is
# near enough to a positive example to affect the random time of the negative
# example.
NEARBY_FIRE_DISTANCE_NO_FIRE_EXAMPLES_KM = 15

# Decimal places to round lat/lon to.
# https://support.garmin.com/en-US/?faq=hRMBoCTy5a7HqVkxukhHd8
LAT_LON_PRECISION = 5
GEOHASH_PRECISION = 7

# Semi-arbitrary
FIRE_HISTORY_NEAR_DISTANCE_KM = 5
FIRE_HISTORY_REGIONAL_DISTANCE_KM = 15
FIRE_HISTORY_RECENT_YEAR_NUM = 3  # TODO: look into finding a specific number
# Arbitrary. Could be as far back as we have data.
FIRE_HISTORY_LONGER_YEAR_NUM = 20

# TODO: Change this to combined fire file once it exists
FIRE_HISTORY_FILEPATH = "/content/drive/My Drive/ML6140  - Project/Raw Data/CombinedFires/combinedFires.csv"
FIRE_HISTORY_DATA = []

CALIFORNIA_GEOJSON_FILE = "/content/drive/My Drive/ML6140  - Project/Raw Data/californiaFromNaturalEarth.json"
CALIFORNIA_POLYGON = None

In [ ]:
def convert_string_to_num_basic_row(row):
  int_keys = ["incident_created_year", "incident_created_month",
              "incident_created_day", "incident_created_hour",
              "incident_created_minute", "class_label"]
  float_keys = ["incident_latitude", "incident_longitude"]
  for key in int_keys:
    try:
      row[BASIC_DATA_HEADER_TO_INDEX[key]] = int(row[BASIC_DATA_HEADER_TO_INDEX[key]])
    except ValueError:
      row[BASIC_DATA_HEADER_TO_INDEX[key]] = float("nan")
  for key in float_keys:
    try:
      row[BASIC_DATA_HEADER_TO_INDEX[key]] = float(row[BASIC_DATA_HEADER_TO_INDEX[key]])
    except ValueError:
      row[BASIC_DATA_HEADER_TO_INDEX[key]] = float("nan")

def read_in_basic_data(filename):
  basic_data = []
  with open(filename, "r", newline="") as infile:
    reader = csv.reader(infile)
    for row in reader:
      convert_string_to_num_basic_row(row)
      basic_data.append(row)
  basic_data = basic_data[1:]  # eliminate header row
  return basic_data

def create_our_dataset(from_existing_basic_data=True):
  """
  from_existing_basic_data: whether or not to read from the existing basic data
    file.
  """
  basic_data = []
  if from_existing_basic_data:
    print("Using existing basic data.")
    basic_data = read_in_basic_data(OUR_DATASET_BASIC_FILENAME)
  else:
    print("Creating new basic data.")
    basic_data = create_basic_example_data()

  # TODO: uncomment
  enhanced_examples = match_date_location_examples_to_weather_and_history(basic_data)
  header = get_enhanced_header()
  out_data = [header] + enhanced_examples
  write_csv(OUR_DATASET_ENHANCED_FILENAME, out_data)
  return enhanced_examples

In [ ]:
def create_basic_example_data():
  """
  Create negative examples and write basic info (time/location/fire) for each
  example to
  """
  positive_examples = read_in_positive_examples()
  negative_examples = create_no_fire_examples()
  if (len(positive_examples)
      and len(BASIC_DATA_FEATURES) != len(positive_examples[0])):
    print(len(BASIC_DATA_FEATURES))
    print(len(positive_examples))
    raise Exception("Number of featers names doesn't match number of features.")
  if (len(negative_examples) and len(positive_examples)
      and len(negative_examples[0]) != len(positive_examples[0])):
    raise Exception("Different number of features for positive and negative examples.")
  combined_examples = positive_examples + negative_examples
  write_csv(OUR_DATASET_BASIC_FILENAME, [BASIC_DATA_FEATURES] + combined_examples)
  print("len(positive_examples): ", len(positive_examples))
  print("len(negative_examples): ", len(negative_examples))
  return combined_examples

## Read in positive examples

In [ ]:
def filter_examples(examples):
  filtered_examples = []
  for row in examples:
    if row[BASIC_DATA_HEADER_TO_INDEX['incident_created_year']] < EXAMPLE_DATA_START_YEAR:
      print("Eliminated row from examples because year is too early: ", row)
      continue
    if (not in_california(row[BASIC_DATA_HEADER_TO_INDEX['incident_latitude']],
                          row[BASIC_DATA_HEADER_TO_INDEX['incident_longitude']])):
      print("Eliminated row from examples because location is not in Califonia: ", row)
      continue
    filtered_examples.append(row)
  return filtered_examples

In [ ]:
def read_in_positive_examples():
  print("Reading in positive examples.")
  positive_examples = []
  with open(CAL_FIRE_FILENAME, "r", newline="") as infile:
    reader = csv.DictReader(infile)
    for row in tqdm(reader):
      positive_examples.append([
          row["incident_name"],
          int(row["incident_created_year"]),
          int(row["incident_created_month"]),
          int(row["incident_created_day"]),
          int(row["incident_created_hour"]),
          int(row["incident_created_minute"]),
          float(row["incident_latitude"]),
          float(row["incident_longitude"]),
          1
      ])
  return filter_examples(positive_examples)

## Create negative examples

In [ ]:
# Approximate. Source: https://observablehq.com/@rdmurphy/u-s-state-bounding-boxes
CA_LAT_BOUNDS = (32.5342307609976, 42.00965914828148)
CA_LON_BOUNDS = (-124.41060660766607, -114.13445790587905)

DATE_RANGE = ((EXAMPLE_DATA_START_YEAR, 1, 1), (2023, 10, 1))

In [ ]:
def convert_string_date_to_int(s):
  try:
    return int(s)
  except ValueError:
    return float("inf")

In [ ]:
def get_positive_example_locations_times():
  examples = []
  with open(CAL_FIRE_FILENAME, "r", newline="") as infile:
    reader = csv.DictReader(infile)
    for row in reader:
      # We care about the day, not the hour and minute for creating negative
      # examples.
      if (convert_string_date_to_int(row["incident_created_year"]) >= EXAMPLE_DATA_START_YEAR
          and in_california(float(row["incident_latitude"]),
                            float(row["incident_longitude"]))):
        examples.append([
            (float(row["incident_latitude"]),
            float(row["incident_longitude"])),
            (convert_string_date_to_int(row["incident_created_year"]),
            convert_string_date_to_int(row["incident_created_month"]),
            convert_string_date_to_int(row["incident_created_day"])),
            (convert_string_date_to_int(row["incident_extinguished_year"]),
            convert_string_date_to_int(row["incident_extinguished_month"]),
            convert_string_date_to_int(row["incident_extinguished_day"]))])
  return examples

In [ ]:
def get_random_time_tuple_in_range():
  # https://stackoverflow.com/questions/553303/generate-a-random-date-between-two-other-dates
  start = datetime(*DATE_RANGE[0])
  end = datetime(*DATE_RANGE[1])
  minutes_between = (end - start).days * 24 * 60
  delta_minutes = random.randint(0, minutes_between)
  random_time = start + timedelta(minutes=delta_minutes)
  as_tuple = (random_time.year, random_time.month, random_time.day, random_time.hour, random_time.minute)
  return as_tuple

In [ ]:
def get_fire_location_matching_examples(num, pos_locations_times):
  examples = []
  for i in tqdm(range(num), desc="Creating matching negative examples"):
    fire = random.choice(pos_locations_times)

    # Get time not during the active fire.
    # TODO: Extend the dates it can't be to include the week/month/year after? here and in random examples
    time = get_random_time_tuple_in_range()
    while time[:3] >= fire[1] and time[:3] < fire[2]:
      time = get_random_time_tuple_in_range()

    # [year, month, day, hour, minute, latitue, longitude, fire true/false]
    examples.append([f"synthetic negative matching {i}", *time, *fire[0], 0])    # TODO: true/false or 1/0?
  return examples

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Difference in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

In [ ]:
def calculate_circular_mean(angles):
    angles_rad = np.radians(angles.dropna())  # Convert to radians and drop NaNs
    sin_sum = np.sum(np.sin(angles_rad))
    cos_sum = np.sum(np.cos(angles_rad))
    return np.degrees(np.arctan2(sin_sum, cos_sum)) % 360

In [ ]:
def within_distance(point1, point2, distance):
  return haversine_distance(*point1, *point2) <= distance

In [ ]:
def get_all_points_within_radius(center, points_and_more_list, radius):
  """
  @param center: (float, float). Must be tuple of len 2
  @param points_and_more_list: [(point, ...), ...] where point = (float, float)
    Tuples must be len 2+
  @param radius: float
  Returns a list containing all of the elements from points_and_more_list where
  the element[0] is a point within a radius' distance from the center.
  Coordinate order must be consistent between points (e.g., if center uses
  (lat, lon), points cannot use (lon, lat))
  """
  filtered = []
  for elem in points_and_more_list:
    if abs(elem[0][0] - center[0]) > radius:
      continue
    if abs(elem[0][1] - center[1]) > radius:
      continue
    if within_distance(center, elem[0], radius):
      filtered.append(elem)
  return filtered

In [ ]:
def within_any_range_in_list(tuple_to_check, list_of_ranges):
  """
  tuple_to_check: float tuple
  list_of_ranges: [(float tuple start, float tuple end), ...]
  """
  for start, end in list_of_ranges:
    if start <= tuple_to_check and tuple_to_check <= end:
      return True

In [ ]:
def load_california_polygon(filepath):
  global CALIFORNIA_POLYGON
  with open("/content/drive/My Drive/ML6140  - Project/Raw Data/californiaFromNaturalEarth.json") as infile:
    cal_geojson = json.load(infile)
    CALIFORNIA_POLYGON = MultiPolygon(cal_geojson['features'][0]["geometry"]["coordinates"])

def in_california(lat, lon):
  if not CALIFORNIA_POLYGON:
    load_california_polygon(CALIFORNIA_GEOJSON_FILE)

  if CALIFORNIA_POLYGON:
    # To compare to polygon from GeoJSON, must also have (lon, lat)
    test_point = Point([lon, lat])
    return test_point.within(CALIFORNIA_POLYGON)
  else:
    raise Exception("There is not california_polygon with which to validate locations.")

In [ ]:
def get_completely_random_examples(num, pos_locations_times):
  # geolocator = Nominatim(user_agent="mlFireProj")
  fire_coords = set([x[0] for x in pos_locations_times])

  examples = []

  for i in tqdm(range(num), desc="Creating random negative examples"):
    # Get random location within the state.
    lat = round(random.uniform(*CA_LAT_BOUNDS), LAT_LON_PRECISION)
    lon = round(random.uniform(*CA_LON_BOUNDS), LAT_LON_PRECISION)
    # geolocated_info = geolocator.reverse(f"{lat}, {lon}")
    # # None if not in state/country land, e.g. Ocean
    # while (not geolocated_info
    #        or "state" not in geolocated_info.raw["address"]
    #        or geolocated_info.raw["address"]["state"] != "California"):
    while not in_california(lat, lon):
      lat = round(random.uniform(*CA_LAT_BOUNDS), LAT_LON_PRECISION)
      lon = round(random.uniform(*CA_LON_BOUNDS), LAT_LON_PRECISION)
      # geolocated_info = geolocator.reverse(f"{lat}, {lon}")

    # Get get time that does not overlap with the time of fires nearby.
    # TODO: lat/lon to miles conversion and distance based on that
    nearby_fires = get_all_points_within_radius(
        (lat, lon), pos_locations_times, NEARBY_FIRE_DISTANCE_NO_FIRE_EXAMPLES_KM)
    blackout_times = [(x[1], x[2]) for x in nearby_fires]  # start/stop times
    time = get_random_time_tuple_in_range()
    while within_any_range_in_list(time, blackout_times):
      time = get_random_time_tuple_in_range()

    examples.append([f"synthetic negative random {i}", *time, lat, lon, 0])

  return examples

In [ ]:
# Create date/time/location data, like Cal Fire data, but fore dates/locations without fires.
# num_matching_fires number of examples at locations of fires in the dataset
# num_random locatations around the state
# all random time
def create_no_fire_examples():
  print("Creating negative examples.")
  # Figure out number of examples (matching and random) to create
  total_examples = 0
  num_matching_fires = 0
  num_completely_random = 0

  positive_example_locations_times = get_positive_example_locations_times()
  total_examples = RATIO_OF_NEGATIVE_EXAMPLES_TO_POSITIVE * len(positive_example_locations_times)
  total_examples = max(0, int(total_examples + .5))  # Round
  num_matching_fires = int(PORTION_OF_NEGATIVE_EXAMPLES_TO_MATCH_FIRE_LOCATIONS * total_examples)
  num_completely_random = total_examples - num_matching_fires
  num_matching_fires = max(0, int(num_matching_fires + .5))
  num_completely_random = max(0, int(num_completely_random + .5))

  matching_examples = get_fire_location_matching_examples(
      num_matching_fires, positive_example_locations_times)

  random_examples = get_completely_random_examples(
      num_completely_random, positive_example_locations_times)

  # based on model performance, we may get rid of the non-random locations
  return matching_examples + random_examples

In [ ]:
# Arrays of locations and times should include fire/no fire boolean

## Complete examples: add weather and history to date/time

match_date_location_examples_to_weather_and_history input: array (cal fire): [incident_name, year, month, day, hour, minute, lat, lon, fire true/false]

output: [
  incident_name,
  year, month, day, hour, minute,
  lat, lon,

  daily_prcp, daily_prcp_dist,  # one day's measurement
  daily_max_temp, daily_max_temp_dist,
  ...,

  agg_prcp, agg_prcp_dist,  (agg_10_prc/agg_30_prc/agg_60_prc)
  ...,

  near_and_recent_acres_burned,
  farther_acres_burned,

  fire true/false
  ]

In [ ]:
# returns an array of complete dataset examples with weather data and fire history data
# for the given array of locations and times

# Return new array (don't modify existing one)
# basic_date: [[fire_name, start_year, start_mont, start_day, start_hour, lat, lon, 1/0]]
def match_date_location_examples_to_weather_and_history(basic_data):  # Array example arrays
  load_fire_history_data()

  enhanced_data = []

  # loop thru array and
  # this func will return a new array with the extra data for the example
  for i in tqdm(range(len(basic_data)), total=len(basic_data)):
    enhanced_row = match_date_location_example_to_weather_and_history(basic_data[i][:-1])  # exludes class label, add appropriate weather and history features to row through the function call
    enhanced_row.append(basic_data[i][-1])  # add class label back in to the enhanced row
    enhanced_data.append(enhanced_row) # append completed, enhanced row to the new enhanced data array

  return enhanced_data

In [ ]:
def get_enhanced_header():
  enhanced_header = BASIC_DATA_FEATURES[:-1]
  for key in FEATURE_INCLUSIONS.keys():
    if FEATURE_INCLUSIONS[key]:
      enhanced_header += FEATURE_HEADERS[key]
  enhanced_header.append(BASIC_DATA_FEATURES[-1])
  return enhanced_header

In [ ]:
# matches
def match_date_location_example_to_weather_and_history(example):
  """
  Returns the example array with the weather and fire history data that match
  the time and location data appended.

  # TODO: confirm order of features.
  """

  # Extract latitude and longitude
  lat = example[BASIC_DATA_HEADER_TO_INDEX["incident_latitude"]]
  lon = example[BASIC_DATA_HEADER_TO_INDEX["incident_longitude"]]

  FEATURE_INCLUSIONS["incident_geohash"] = True
  get_incident_geohash(example, GEOHASH_PRECISION)


  # Find the nearest GHCN and LCD stations and their distances
  closest_lcd_station, lcd_distance = find_closest_lcd_station(lat, lon)
  closest_ghcn_station, ghcn_distance = find_closest_ghcn_station(lat, lon)

  FEATURE_INCLUSIONS["station_details"] = True
  get_station_details(example, closest_lcd_station, lcd_distance, closest_ghcn_station, ghcn_distance)

  # Fetch and add daily weather data, you said pass the array into these functions and dont return anything
  # so at the end of this function we will return the basic example again.
  FEATURE_INCLUSIONS["daily_weather"] = True
  _, station_tracker = get_daily_weather(example, closest_lcd_station, closest_ghcn_station, lcd_distance, ghcn_distance)


  # Fetch and add aggregate weather data for different time windows
  FEATURE_INCLUSIONS["agg_10_weather"] = True
  get_aggregate_weather(example, closest_lcd_station, closest_ghcn_station, station_tracker, 10)
  FEATURE_INCLUSIONS["agg_30_weather"] = True
  get_aggregate_weather(example, closest_lcd_station, closest_ghcn_station, station_tracker, 30)
  FEATURE_INCLUSIONS["agg_60_weather"] = True
  get_aggregate_weather(example, closest_lcd_station, closest_ghcn_station, station_tracker, 60)

  # small_are_num_acres_burned (e.g. 50 mi, last 3 years)
  # larger_area_time_num_acres (e.g. 100 mi, last 15 years)
  FEATURE_INCLUSIONS["fire_history"] = True
  get_fire_history_long_and_short(example)

  return example

In [ ]:
# For each weather measurement, include distance to measurement.
# e.g. prcp, prcp_distance, max_temp, max_temp_distance, etc.

In [ ]:
def convert_hist_csv_row_dict_to_example(row):
  int_keys = ['incident_created_year', 'incident_created_month',
               'incident_created_day', 'incident_created_hour',
               'incident_created_minute', 'incident_extinguished_year',
               'incident_extinguished_month', 'incident_extinguished_day',
               'incident_extinguished_hour', 'incident_extinguished_minute']
  float_keys = ['incident_acres_burned', 'incident_longitude',
                 'incident_latitude']
  for key in int_keys:
    try:
      row[key] = int(row[key])
    except ValueError:
      row[key] = float("nan")
  for key in float_keys:
    try:
      row[key] = float(row[key])
    except ValueError:
      row[key] = float("nan")
  return row

def load_fire_history_data():
  global FIRE_HISTORY_DATA
  FIRE_HISTORY_DATA = []
  with open(FIRE_HISTORY_FILEPATH, "r", newline="") as infile:
    reader = csv.DictReader(infile)
    for row in reader:
      FIRE_HISTORY_DATA.append(convert_hist_csv_row_dict_to_example(row))
  FIRE_HISTORY_DATA.sort(key=lambda x: (x["incident_latitude"], x["incident_longitude"]))

In [ ]:
def get_incident_geohash(example, precision=7):
    lat = example[BASIC_DATA_HEADER_TO_INDEX["incident_latitude"]]
    lon = example[BASIC_DATA_HEADER_TO_INDEX["incident_longitude"]]

    # Check if latitude and longitude are valid
    if lat is not None and lon is not None:
        try:
            # Check if lat and lon are within valid ranges
            if -90 <= lat <= 90 and -180 <= lon <= 180:
                tile_hash = geohash.encode(lat, lon, precision)
                example.append(tile_hash)
            else:
                # Append NaN if lat or lon is out of range
                example.append(np.nan)
        except Exception as e:
            print(f"Error encoding geohash: {e}")
            # Append NaN if there's an error during geohash encoding
            example.append(np.nan)
    else:
        # Append NaN if lat or lon is missing
        example.append(np.nan)


In [ ]:
def get_station_details(example, closest_lcd_station, lcd_distance, closest_ghcn_station, ghcn_distance):
    lcd_station_id, lcd_elevation, ghcn_station_id, ghcn_elevation = np.nan, np.nan, np.nan, np.nan

    if closest_lcd_station is not None:
        lcd_station_id = str(closest_lcd_station.get('STATION_ID', '')).zfill(5) if closest_lcd_station.get('STATION_ID', '') else np.nan
        lcd_elevation_str = closest_lcd_station.get('STATION_ELEVATION', '')

        try:
            lcd_elevation = float(lcd_elevation_str.rstrip('m').strip())

        except ValueError:
            lcd_elevation = np.nan

    if closest_ghcn_station is not None:
        ghcn_station_id = closest_ghcn_station.get('STATION_ID', np.nan)
        ghcn_elevation = closest_ghcn_station.get('ELEVATION', np.nan)

    example.extend([lcd_station_id, lcd_elevation, lcd_distance, ghcn_station_id, ghcn_elevation, ghcn_distance])


In [ ]:
def get_daily_weather(example, lcd_station, ghcn_station, lcd_distance, ghcn_distance):

  year, month, day = example[1:4]

  # Load data from the stations folders
  lcd_station_id_str = str(lcd_station['STATION_ID']).zfill(5)
  lcd_data = pd.read_csv(f"{LCD_STATIONS_FOLDER}/{lcd_station_id_str}.csv")
  ghcn_data = pd.read_csv(f"{GHCN_STATIONS_FOLDER}/{ghcn_station['STATION_ID']}.csv")

  # Filter data for the specific date
  lcd_daily = lcd_data[(lcd_data['YEAR'] == year) & (lcd_data['MONTH'] == month) & (lcd_data['DAY'] == day)]
  ghcn_daily = ghcn_data[(ghcn_data['YEAR'] == year) & (ghcn_data['MONTH'] == month) & (ghcn_data['DAY'] == day)]

  station_tracker = {}

  # Process Shared Features
  shared_feature_mapping = {
      'PRCP': 'sumPrecipitation',
      'TMAX': 'maxDryBulbTemperature',
      'TMIN': 'minDryBulbTemperature',
      'TAVG': 'meanDryBulbTemperature',
      'ADPT': 'meanDewPointTemperature',
      'AWBT': 'meanWetBulbTemperature',
      'AWND': 'meanWindSpeed',
      'RHAV': 'meanRelativeHumidity',
      'RHMN': 'minRelativeHumidity',
      'RHMX': 'maxRelativeHumidity'
  }

  for ghcn_feature, lcd_feature in shared_feature_mapping.items():
    lcd_value = lcd_daily[lcd_feature].iloc[0] if not lcd_daily.empty else np.nan
    ghcn_value = ghcn_daily[ghcn_feature].iloc[0] if not ghcn_daily.empty else np.nan

    # Check which station is closer and try to pull data from it
    if lcd_distance <= ghcn_distance:
        chosen_value, chosen_distance = (lcd_value, lcd_distance) if not pd.isna(lcd_value) else (ghcn_value, ghcn_distance)
    else:
        chosen_value, chosen_distance = (ghcn_value, ghcn_distance) if not pd.isna(ghcn_value) else (lcd_value, lcd_distance)

    # Set to NaN only if both stations have missing data
    if pd.isna(chosen_value):
        chosen_distance = np.nan

    example.extend([chosen_value, chosen_distance])

    # Determine which station's data was used
    if chosen_distance == lcd_distance:
      station_tracker[lcd_feature] = 'LCD'
    elif chosen_distance == ghcn_distance:
      station_tracker[ghcn_feature] = 'GHCN'
    else:
      station_tracker[lcd_feature] = 'None'



  # Process Unique GHCN Features
  for feature in ['SNOW', 'SNWD', 'EVAP', 'FMTM', 'FRGB', 'FRGT', 'FRTH']:
    value = ghcn_daily[feature].iloc[0] if not ghcn_daily.empty else np.nan
    distance = ghcn_distance if not pd.isna(value) else np.nan
    example.extend([value, distance])

    station_tracker[feature] = 'GHCN' if not pd.isna(value) else 'None'

  # Process Unique LCD Features
  for feature in ['maxWindSpeed', 'calculate_circular_meanWindDirection', 'mode_functionWindDirection', 'maxWindGustSpeed', 'mode_functionPresentWeatherType', 'mode_functionSkyConditions']:
    value = lcd_daily[feature].iloc[0] if not lcd_daily.empty else np.nan
    distance = lcd_distance if not pd.isna(value) else np.nan
    example.extend([value, distance])

    station_tracker[feature] = 'LCD' if not pd.isna(value) else 'None'

  return example, station_tracker


In [ ]:
def get_aggregate_weather(example, lcd_station, ghcn_station, station_tracker, num_days, threshold=0.6):
    year, month, day, hour = example[1:5]
    end_date = datetime(year, month, day)

    # Adjust the end date based on the time of the incident
    if hour < 12:
        # If the incident is before noon, exclude the incident day
        end_date = end_date - timedelta(days=1)

    start_date = end_date - timedelta(days=num_days - 1)


    # Load data for the closest LCD and GHCN stations
    lcd_station_id_str = str(lcd_station['STATION_ID']).zfill(5)
    lcd_data = pd.read_csv(f"{LCD_STATIONS_FOLDER}/{lcd_station_id_str}.csv")
    ghcn_data = pd.read_csv(f"{GHCN_STATIONS_FOLDER}/{ghcn_station['STATION_ID']}.csv")


    lcd_data['DATE'] = pd.to_datetime(lcd_data[['YEAR', 'MONTH', 'DAY']])
    ghcn_data['DATE'] = pd.to_datetime(ghcn_data[['YEAR', 'MONTH', 'DAY']])

    # Filter data for the specific date range
    lcd_filtered = lcd_data[(lcd_data['DATE'] >= start_date) & (lcd_data['DATE'] <= end_date)]
    ghcn_filtered = ghcn_data[(ghcn_data['DATE'] >= start_date) & (ghcn_data['DATE'] <= end_date)]


    aggregated_data = []

    # Aggregate data for each feature based on station_tracker
    for feature, station in station_tracker.items():
        if station == 'LCD':
            if not is_data_sufficient(lcd_filtered[feature], threshold):
                aggregated_value = np.nan
            else:
                aggregated_value = aggregate_feature(lcd_filtered[feature], feature, num_days)
        elif station == 'GHCN':
            if not is_data_sufficient(ghcn_filtered[feature], threshold):
                aggregated_value = np.nan
            else:
                aggregated_value = aggregate_feature(ghcn_filtered[feature], feature, num_days)
        else:
            aggregated_value = np.nan  # No data available

        aggregated_data.append(aggregated_value)

    example.extend(aggregated_data)
    return example

def is_data_sufficient(data, threshold):
    if len(data) == 0:
        return False
    proportion_non_nan = data.notna().sum() / len(data)
    return proportion_non_nan >= threshold


def aggregate_feature(data, feature_name, num_days):
    if data.empty:
        return np.nan
    elif feature_name == 'calculate_circular_meanWindDirection':
        return calculate_circular_mean(data)
    elif feature_name in ['mode_functionWindDirection', 'mode_functionPresentWeatherType', 'mode_functionSkyConditions', 'FMTM']:
        mode_value = data.mode()
        return mode_value[0] if not mode_value.empty else np.nan
    else:
        return data.mean()


In [ ]:
# From Cal Fire and Kaggle
def get_fire_history_long_and_short(example):
  INCIDENT_NAME_INDEX = BASIC_DATA_HEADER_TO_INDEX["incident_name"]
  INCIDENT_CREATED_YEAR_INDEX = BASIC_DATA_HEADER_TO_INDEX["incident_created_year"]
  INCIDENT_CREATED_MONTH_INDEX = BASIC_DATA_HEADER_TO_INDEX["incident_created_month"]
  INCIDENT_CREATED_DAY_INDEX = BASIC_DATA_HEADER_TO_INDEX["incident_created_day"]
  INCIDENT_CREATED_MINUTE_INDEX = BASIC_DATA_HEADER_TO_INDEX["incident_created_minute"]
  INCIDENT_LATITUDE_INDEX = BASIC_DATA_HEADER_TO_INDEX["incident_latitude"]
  INCIDENT_LONGITUDE_INDEX = BASIC_DATA_HEADER_TO_INDEX["incident_longitude"]

  lat = example[INCIDENT_LATITUDE_INDEX]
  lon = example[INCIDENT_LONGITUDE_INDEX]

  try:
    date = datetime(*example[INCIDENT_CREATED_YEAR_INDEX:INCIDENT_CREATED_MINUTE_INDEX + 1])
  except ValueError:
    try:
      date = datetime(*example[INCIDENT_CREATED_YEAR_INDEX:INCIDENT_CREATED_DAY_INDEX + 1])
    except ValueError as err:
      print("Warning: Invalid example date.")
      print("Example: ", example)
      print("Err: ", err)
      example += [float("nan"), float("nan")]
      return

  try:
    short_range_start_date = date.replace(year=date.year - FIRE_HISTORY_RECENT_YEAR_NUM)
    long_range_start_date = date.replace(year=date.year - FIRE_HISTORY_LONGER_YEAR_NUM)
  except ValueError as err:
    if str(err) == "day is out of range for month":
      # Leap year. 2004, 2, 29 is valid, but 2001, 2, 29 is not.
      short_range_start_date = date.replace(year=date.year - FIRE_HISTORY_RECENT_YEAR_NUM, day=28)
      long_range_start_date = date.replace(year=date.year - FIRE_HISTORY_LONGER_YEAR_NUM, day=28)
    else:
      print("Warning: Unable to create previous dates for example date.")
      print("Example: ", example)
      print("Err: ", err)
      example += [float("nan"), float("nan")]
      return

  far_fires_acres = []
  near_fires_acres = []

  def bin_search_first(target_lat_lon, max_dist):
    start_idx = 0
    end_idx = len(FIRE_HISTORY_DATA) - 1
    while start_idx < end_idx:
      curr = (start_idx + end_idx) // 2
      # Search for farthest latitude within range

      in_range = within_distance(target_lat_lon,
                      (FIRE_HISTORY_DATA[curr]["incident_latitude"], target_lat_lon[1]),
                      max_dist)
      if in_range or FIRE_HISTORY_DATA[curr]["incident_latitude"] > target_lat_lon[0]:
        end_idx = curr
      else:
        start_idx = curr + 1
    return start_idx

  def bin_search_last(target_lat_lon, max_dist):
    start_idx = 0
    end_idx = len(FIRE_HISTORY_DATA) - 1
    while start_idx < end_idx:
      curr = (start_idx + end_idx) // 2 + 1
      # Search for farthest latitude within range
      in_range = within_distance(target_lat_lon,
                      (FIRE_HISTORY_DATA[curr]["incident_latitude"], target_lat_lon[1]),
                      max_dist)
      if in_range or FIRE_HISTORY_DATA[curr]["incident_latitude"] < target_lat_lon[0]:
        start_idx = curr
      else:
        end_idx = curr - 1
    return end_idx

  start_idx = bin_search_first((lat, lon), FIRE_HISTORY_REGIONAL_DISTANCE_KM)
  end_idx = bin_search_last((lat, lon), FIRE_HISTORY_REGIONAL_DISTANCE_KM)
  for i in range(start_idx, end_idx + 1):
    # Skip over this incident itself
    if (FIRE_HISTORY_DATA[i]["incident_name"] == example[INCIDENT_NAME_INDEX]
        and FIRE_HISTORY_DATA[i]["incident_created_year"] == example[INCIDENT_CREATED_YEAR_INDEX]
        and FIRE_HISTORY_DATA[i]["incident_created_month"] == example[INCIDENT_CREATED_MONTH_INDEX]
        and FIRE_HISTORY_DATA[i]["incident_created_day"] == example[INCIDENT_CREATED_DAY_INDEX]
        and FIRE_HISTORY_DATA[i]["incident_latitude"] == example[INCIDENT_LATITUDE_INDEX]
        and FIRE_HISTORY_DATA[i]["incident_longitude"] == example[INCIDENT_LONGITUDE_INDEX]):
      continue

    # Ideally judge nearness by date extinguished, but can use start date
    # since many don't have a date extinguished
    try:
      hist_ext_date = datetime(FIRE_HISTORY_DATA[i]["incident_extinguished_year"],
                              FIRE_HISTORY_DATA[i]["incident_extinguished_month"],
                              FIRE_HISTORY_DATA[i]["incident_extinguished_day"])
    except TypeError:
      if (np.isnan(FIRE_HISTORY_DATA[i]["incident_extinguished_year"])
          or np.isnan(FIRE_HISTORY_DATA[i]["incident_extinguished_month"])
          or np.isnan(FIRE_HISTORY_DATA[i]["incident_extinguished_day"])):
        hist_ext_date = datetime(FIRE_HISTORY_DATA[i]["incident_created_year"],
                            FIRE_HISTORY_DATA[i]["incident_created_month"],
                            FIRE_HISTORY_DATA[i]["incident_created_day"])
      else:
        print("err", i)

    if (long_range_start_date <= hist_ext_date and hist_ext_date < date  # TODO: buffer before curr date?
        and within_distance((lat, lon),
                            (FIRE_HISTORY_DATA[i]["incident_latitude"],
                             FIRE_HISTORY_DATA[i]["incident_longitude"]),
                            FIRE_HISTORY_REGIONAL_DISTANCE_KM)):
      far_fires_acres.append(FIRE_HISTORY_DATA[i]["incident_acres_burned"])
      if (short_range_start_date <= hist_ext_date
          and within_distance((lat, lon),
                            (FIRE_HISTORY_DATA[i]["incident_latitude"],
                             FIRE_HISTORY_DATA[i]["incident_longitude"]),
                            FIRE_HISTORY_NEAR_DISTANCE_KM)):
        near_fires_acres.append(FIRE_HISTORY_DATA[i]["incident_acres_burned"])

  # Use yearly average as a feature in case different len histories
  example.append(sum(far_fires_acres) / FIRE_HISTORY_LONGER_YEAR_NUM)
  example.append(sum(near_fires_acres) / FIRE_HISTORY_RECENT_YEAR_NUM)

In [ ]:
def find_closest_ghcn_station(lat, lon):

    # Define the column names since the file doesn't have headers
    column_names = ['STATION_ID', 'LATITUDE', 'LONGITUDE', 'ELEVATION']
    # Read the station data from the CSV file, assuming no header
    ghcn_station_data = pd.read_csv(GHCN_STATIONS_FILE, header=None, names=column_names)

    closest_station = None
    min_distance = float('inf')

    # Iterate through each station and calculate the haversine distance
    for index, station in ghcn_station_data.iterrows():
        dist = haversine_distance(lat, lon, station['LATITUDE'], station['LONGITUDE'])
        if dist < min_distance:
            min_distance = dist
            closest_station = station

    return closest_station, min_distance

In [ ]:
def find_closest_lcd_station(lat, lon):
    # Static file path for LCD station data
    lcd_station_data = pd.read_csv(LCD_STATIONS_FILE)

    closest_station = None
    min_distance = float('inf')
    # Iterate through each station and calculate the haversine distance
    for index, station in lcd_station_data.iterrows():
        dist = haversine_distance(lat, lon, station['STATION_LATITUDE'], station['STATION_LONGITUDE'])
        if dist < min_distance:
            min_distance = dist
            closest_station = station

    return closest_station, min_distance

## Write out new dataset

In [ ]:
def write_csv(filename, data_array):
  print("\nWriting to file ", filename)
  with open(filename, "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerows(data_array)

## Run

In [ ]:
res = create_our_dataset(from_existing_basic_data=False)

Creating new basic data.
Reading in positive examples.


2180it [00:00, 3777.99it/s]


Eliminated row from examples because year is too early:  ['River Fire', 2013, 2, 24, 8, 16, 36.602575, -118.01651, 1]
Eliminated row from examples because year is too early:  ['Fawnskin Fire', 2013, 4, 20, 17, 30, 34.288877, -116.941311, 1]
Eliminated row from examples because year is too early:  ['Gold Fire', 2013, 4, 30, 12, 59, 37.116295, -119.635004, 1]
Eliminated row from examples because year is too early:  ['Silverado Fire', 2013, 4, 30, 23, 44, 38.441792, -122.350844, 1]
Eliminated row from examples because year is too early:  ['Yellow Fire', 2013, 5, 1, 2, 1, 38.638828, -122.655616, 1]
Eliminated row from examples because year is too early:  ['Panther Fire', 2013, 5, 1, 9, 12, 40.190062, -121.595555, 1]
Eliminated row from examples because year is too early:  ['Summit Fire', 2013, 5, 1, 12, 38, 34.288877, -116.941311, 1]
Eliminated row from examples because year is too early:  ['306 Fire', 2013, 5, 1, 19, 0, 39.514139, -122.560862, 1]
Eliminated row from examples because year 

Creating random negative examples: 100%|██████████| 2909/2909 [00:15<00:00, 183.39it/s]



Writing to file  /content/drive/My Drive/ML6140  - Project/Data/basic_data.csv
len(positive_examples):  1939
len(negative_examples):  3878


100%|██████████| 5817/5817 [21:36<00:00,  4.49it/s]



Writing to file  /content/drive/My Drive/ML6140  - Project/Data/enhanced_data.csv
